In [1]:
import json
import pandas as pd

In [3]:
f = open('/home/dy/filerepo/Research/multiagent-human-experiment/mturk/experiment_data/mturk-experiment-5-20210827-bonus-flipped.json',)
data = json.load(f)
f.close()

feedback_subtasks = [5, 7, 9, 11]
game_subtasks = [4, 6, 8, 10]
workers = range(len(data))
agents = [game[1] for game in data[0]['results']['data'][-1]['games']]
print(agents)

['sac_self_play_coordination_ring_0', 'nct_coordination_ring', 'action_cml_v_sac_none_no_ib', 'sac_v_bc_states_coordination_ring_0']


In [4]:
all_data = pd.DataFrame()
df = pd.DataFrame()


responses = []
for worker in workers:
    for agent in agents:
        for game_subtask in game_subtasks: #find the right subtask for the agent order
            # if (data[worker]['results']['data'][game_subtask]['data']['trajectory'][0]['player_1_id'] == agent):
            if (data[worker]['results']['data'][game_subtask]['params']['playerOne'] == agent):
                game_info = data[worker]['results']['data'][game_subtask+1]
                game_info['agent'] = agent
                game_info['workerId'] = data[worker]['workerId']
                responses.append(data[worker]['results']['data'][game_subtask+1])

df = pd.DataFrame.from_dict(responses)
df

,helpful,competent,enjoy,humanlike,efficient,flexible,predictable,partnerFeedback,agent,workerId
0,6,7,6,7,7,6,7,Nice,sac_self_play_coordination_ring_0,A32W5S5JWI020D
1,6,6,7,7,6,7,6,Nice,nct_coordination_ring,A32W5S5JWI020D
2,6,6,7,6,7,6,6,Nice,action_cml_v_sac_none_no_ib,A32W5S5JWI020D
3,6,7,6,6,7,6,7,Nice,sac_v_bc_states_coordination_ring_0,A32W5S5JWI020D
4,7,7,6,5,6,6,7,very well,sac_self_play_coordination_ring_0,A8QQAWKPF8INN
...,...,...,...,...,...,...,...,...,...,...
75,4,5,4,4,5,4,4,Good,sac_v_bc_states_coordination_ring_0,A1BISRD579W10W
76,6,7,7,6,6,7,4,GOOD,sac_self_play_coordination_ring_0,A152NBCP5LTKJF
77,6,7,6,7,6,6,7,VERY NICE,nct_coordination_ring,A152NBCP5LTKJF
78,6,7,6,7,4,6,6,VERY NICE,action_cml_v_sac_none_no_ib,A152NBCP5LTKJF


In [31]:
def get_subtask(agent, worker, game_subtasks, data):

    for game_subtask in game_subtasks: #find the right subtask for the agent order
        # if (data[worker]['results']['data'][game_subtask]['data']['trajectory'][0]['player_1_id'] == agent):
        if (data[worker]['results']['data'][game_subtask]['params']['playerOne'] == agent):
            return game_subtask
    return -1

def get_breakdown(worker, game_subtask, data):
    game_name = data[worker]['results']['data'][game_subtask]['params']['playerOne']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/mturk/experiment_data/experiment-5-agent-data/mturk_replay_worker' + str(worker) + '_' + game_name + '.json')
    agent_data = json.load(f)    
    f.close()

    agent_0_soups_delivered = agent_data.get('_agent_0_soups_delivered', 0)
    agent_1_soups_delivered = agent_data.get('_agent_1_soups_delivered', 0)

    agent_0_onions_placed_in_pot = agent_data.get('_agent_0_onions_placed_in_pot', 0)
    agent_1_onions_placed_in_pot = agent_data.get('_agent_1_onions_placed_in_pot', 0)

    agent_0_onion_soups_picked_from_pot = agent_data.get('_agent_0_onion_soups_picked_from_pot', 0)
    agent_1_onion_soups_picked_from_pot = agent_data.get('_agent_1_onion_soups_picked_from_pot', 0)

    breakdown = {
        'worker': worker,
        'agent': game_name,
        'agent_0_soups_delivered': agent_0_soups_delivered,
        'agent_1_soups_delivered': agent_1_soups_delivered,
        'agent_0_onions_placed_in_pot': agent_0_onions_placed_in_pot,
        'agent_1_onions_placed_in_pot': agent_1_onions_placed_in_pot,
        'agent_0_onion_soups_picked_from_pot': agent_0_onion_soups_picked_from_pot,
        'agent_1_onion_soups_picked_from_pot': agent_1_onion_soups_picked_from_pot,
    }

    return breakdown

In [38]:
all_breakdowns = []
for worker in workers:
    for agent in agents:
        game_subtask = get_subtask(agent, worker, game_subtasks, data)
        breakdown = get_breakdown(worker, game_subtask, data)
        all_breakdowns.append(breakdown)

breakdown_df = pd.DataFrame(all_breakdowns)
breakdown_df

,worker,agent,agent_0_soups_delivered,agent_1_soups_delivered,agent_0_onions_placed_in_pot,agent_1_onions_placed_in_pot,agent_0_onion_soups_picked_from_pot,agent_1_onion_soups_picked_from_pot
0,0,sac_self_play_coordination_ring_0,0.0,1.0,4.0,3.0,1.0,1.0
1,0,nct_coordination_ring,0.0,2.0,6.0,3.0,0.0,2.0
2,0,action_cml_v_sac_none_no_ib,1.0,1.0,1.0,5.0,1.0,1.0
3,0,sac_v_bc_states_coordination_ring_0,0.0,2.0,4.0,5.0,0.0,2.0
4,1,sac_self_play_coordination_ring_0,0.0,2.0,4.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...
75,18,sac_v_bc_states_coordination_ring_0,2.0,0.0,3.0,4.0,2.0,0.0
76,19,sac_self_play_coordination_ring_0,0.0,1.0,2.0,4.0,0.0,1.0
77,19,nct_coordination_ring,1.0,1.0,5.0,3.0,1.0,1.0
78,19,action_cml_v_sac_none_no_ib,0.0,3.0,5.0,7.0,0.0,3.0


In [47]:
agent_meta_breakdowns = {}
for agent in agents:
    agent_meta_breakdowns[agent] = breakdown_df[breakdown_df['agent'] == agent].describe()
print(agents)

['sac_self_play_coordination_ring_0', 'nct_coordination_ring', 'action_cml_v_sac_none_no_ib', 'sac_v_bc_states_coordination_ring_0']


In [52]:
agent_meta_breakdowns['sac_v_bc_states_coordination_ring_0']

,worker,agent_0_soups_delivered,agent_1_soups_delivered,agent_0_onions_placed_in_pot,agent_1_onions_placed_in_pot,agent_0_onion_soups_picked_from_pot,agent_1_onion_soups_picked_from_pot
count,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,9.50000,0.600000,2.800000,6.500000,5.600000,0.600000,2.900000
std,5.91608,0.940325,2.117595,4.370957,2.233713,0.940325,2.221901
min,0.00000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
25%,4.75000,0.000000,1.000000,3.000000,4.000000,0.000000,1.000000
50%,9.50000,0.000000,3.000000,5.000000,5.000000,0.000000,3.000000
75%,14.25000,1.000000,4.000000,8.250000,6.000000,1.000000,4.000000
max,19.00000,3.000000,7.000000,16.000000,13.000000,3.000000,8.000000
